# What To Do
CheckM version 1.1.2 <br>
GTDB-TK v1.4.0

https://merenlab.org/2016/06/18/importing-taxonomy/ <br>
https://merenlab.org/2016/06/18/importing-functions/#prokka <br>
https://waoverholt.com/current-MAG-protocol/#taxonomic-annotation-of-bins <br>
https://ecogenomics.github.io/GTDBTk/examples/classify_wf.html <br>
https://merenlab.org/2016/06/22/anvio-tutorial-v2/

# Gene prediction and Annotation
- find ORFs from MAGs <br>
- map reads to ORFs
- import bam file to R
- get RPKM values
- do deseq

[1] eggNOG-mapper v2: functional annotation, orthology assignments, and domain 
      prediction at the metagenomic scale. Carlos P. Cantalapiedra, 
      Ana Hernandez-Plaza, Ivica Letunic, Peer Bork, Jaime Huerta-Cepas. 2021.
      Molecular Biology and Evolution, msab293, https://doi.org/10.1093/molbev/msab293

[2] eggNOG 5.0: a hierarchical, functionally and phylogenetically annotated
      orthology resource based on 5090 organisms and 2502 viruses. Jaime
      Huerta-Cepas, Damian Szklarczyk, Davide Heller, Ana Hernandez-Plaza,
      Sofia K Forslund, Helen Cook, Daniel R Mende, Ivica Letunic, Thomas
      Rattei, Lars J Jensen, Christian von Mering and Peer Bork. Nucleic Acids
      Research, Volume 47, Issue D1, 8 January 2019, Pages D309-D314,
      https://doi.org/10.1093/nar/gky1085 

[3] Sensitive protein alignments at tree-of-life scale using DIAMOND.
       Buchfink B, Reuter K, Drost HG. 2021.
       Nature Methods 18, 366–368 (2021). https://doi.org/10.1038/s41592-021-01101-x

[4] Prodigal: prokaryotic gene recognition and translation initiation site identification.
       Hyatt et al. 2010. BMC Bioinformatics 11, 119. https://doi.org/10.1186/1471-2105-11-119.

e.g. Functional annotation was performed using eggNOG-mapper (version emapper-2.1.10) [1]
 based on eggNOG orthology data [2]. Sequence searches were performed using [3]. Gene prediction was performed using [4].

In [ ]:
conda deactivate
conda activate emapper-gff


mkdir 08_annotation_trimmed/eggnog

for bin in $(find 05_binning/aa_FINAL_04_TRIMMED_CONFIRM/ -name "*.fa" -type f); do
    file_name=$(basename $bin)
    bin_name=${file_name%.*}
    
    emapper.py -i $bin \
    --itype genome --genepred prodigal --cpu 32 \
     -m diamond --data_dir $EGGNOG_DATA_DIR \
    --output $bin_name --output_dir 08_annotation_trimmed/eggnog --decorate_gff yes --override
done


# MagicLamp
Get iron & sulfur metabolism genes

In [ ]:
conda deactivate
conda activate binning

MagicLamp.py FeGenie -bin_dir 08_annotation_trimmed/eggnog/ -bin_ext fasta -out 08_annotation_trimmed/eggnog_fegenieML --orfs --meta --all_results -t 32
MagicLamp.py LithoGenie -bin_dir 08_annotation_trimmed/eggnog/ -bin_ext fasta -out 08_annotation_trimmed/eggnog_lithogenieML --orfs --meta --all_results -t 32
MagicLamp.py LithoGenie -bin_dir 08_annotation_trimmed/eggnog/ -bin_ext fasta -out 08_annotation_trimmed/eggnog_lithogenieML --orfs --meta --all_results -t 32 -cat sulfur --skip



# Mapping
## Reads to Contigs

In [ ]:
# combine MAGs

cat 05_binning/aa_FINAL_04_TRIMMED_CONFIRM/*.fa > 08_annotation_trimmed/FINAL_MAGs.fasta

mkdir 08_annotation_trimmed/eggnog_mapped #eggnog mapped
#conda deactivate
# map reads per sample to genes
for barcode in $(find /data/WINONA/202208_exxon_fastq/ -maxdepth 1 -name "sample*" -type f); do
    file_name=$(basename $barcode)
    sample_name=${file_name%.*}
    MAGs=08_annotation_trimmed/FINAL_MAGs.fasta

    mapped=./08_annotation_trimmed/eggnog_mapped/${sample_name}_mapped.sam
    bam_raw=./08_annotation_trimmed/eggnog_mapped/${sample_name}_raw.bam
    bam_sorted=./08_annotation_trimmed/eggnog_mapped/${sample_name}_sorted.bam
    bam_indexed=./08_annotation_trimmed/eggnog_mapped/${sample_name}_sorted.bam.bai
    
    echo $sample_name

    minimap2 -ax map-ont $MAGs $barcode -t 64 > $mapped
    samtools view -o $bam_raw -F 4 -@ 64 -bS $mapped 
    samtools sort -o $bam_sorted -@ 64 $bam_raw 
    samtools index -b -@ 64 $bam_sorted $bam_indexed 
done


## Get each gene's counts from gff

In [ ]:
# feature counts from gff & bam
# https://youngleebbs.gitbook.io/bioinfo-training/part-ii/2.-construction-of-expression-matrix#2.2-quantify-gene-expression
# featureCounts v2.0.1

conda deactivate
conda activate binning

for annot in $(find 08_annotation_trimmed/eggnog/ -name "*genepred.gff" -type f); do
    file_name=$(basename $annot)
    bin_name=${file_name%.*}
    bin_name=${bin_name%.*}
    bin_name=${bin_name%.*}

    output=08_annotation_trimmed/eggnog_mapped/$bin_name-counts

    featureCounts -t CDS -g 'ID' -T 32 -L -f -O \
        -a $annot \
        -o $output \
        08_annotation_trimmed/eggnog_mapped/sample*_sorted.bam
done


# -f     Perform read counting at feature level (eg. counting reads for exons rather than genes).
# -O     Assign reads to all their overlapping meta-features (or features if -f is specified).
# -M     Multi-mapping reads will also be counted. For a multi-mapping read, all its reported alignments will be counted. The 'NH' tag in BAM/SAM input is used to detect multi-mapping reads.
# -L     Count long reads such as Nanopore and PacBio reads. Long read counting can only run in one thread and only reads (not read-pairs) can be counted. There is no limitation on the number of 'M' operations allowed in a CIGAR string in long read counting.


# CoverM
Get coverage files
https://github.com/wwood/CoverM#calculation-methods <br>
https://wwood.github.io/CoverM/coverm-genome.html#coverage-calculation-options


In [ ]:
mkdir 08_annotation_trimmed/final_TRIMMED_bins_coverage/
coverm genome \
    --bam-files 08_annotation_trimmed/eggnog_mapped/sample*_sorted.bam \
    --genome-fasta-directory 05_binning/aa_FINAL_04_TRIMMED_CONFIRM/ \
    --methods relative_abundance -x fa -t 32 \
    --output-file 08_annotation_trimmed/final_TRIMMED_bins_coverage/relative_abundance.tsv

coverm genome \
    --bam-files 08_annotation_trimmed/eggnog_mapped/sample*_sorted.bam \
    --genome-fasta-directory 05_binning/aa_FINAL_04_TRIMMED_CONFIRM/ \
    --methods variance -x fa -t 32 \
    --output-file 08_annotation_trimmed/final_TRIMMED_bins_coverage/variance.tsv

coverm genome \
    --bam-files 08_annotation_trimmed/eggnog_mapped/sample*_sorted.bam \
    --genome-fasta-directory 05_binning/aa_FINAL_04_TRIMMED_CONFIRM/ \
    --methods mean -x fa -t 32 \
    --output-file 08_annotation_trimmed/final_TRIMMED_bins_coverage/mean.tsv



# On DSM10523

In [ ]:
# annotation
conda deactivate
conda activate emapper-gff

emapper.py -i 09_MAGcompare/Desulfocapsa_sulfexigens_DSM10523_GCF_000341395.fasta \
    --itype genome --genepred prodigal --cpu 32 \
     -m diamond --data_dir $EGGNOG_DATA_DIR \
    --output DSM10523 --output_dir 09_MAGcompare --decorate_gff yes

# FeGenie
conda deactivate
conda activate binning
mkdir 09_MAGcompare/DSM10523_fegenie
FeGenie.py -bin_dir 09_MAGcompare/ -bin_ext fasta -out 09_MAGcompare/DSM10523_fegenie --orfs --meta --all_results --heme --hematite --makeplots -t 16


# Find HGT genes
using HGTector2

In [ ]:
conda activate hgtector

hgtector search -i 08_annotation_trimmed/eggnog/binsanity_sample06.txt.002.emapper.genepred.fasta -o 08_annotation_trimmed/eggnog_HGTector -m diamond -p 32 \
    -d /data/WINONA/common_bioinformatics/database/hgtdb_20230102/diamond/db.dmnd \
    -t /data/WINONA/common_bioinformatics/database/hgtdb_20230102/taxdump

hgtector analyze \
    -i 08_annotation_trimmed/eggnog_HGTector/search \
    -o 08_annotation_trimmed/eggnog_HGTector/analyse \
    -t /data/WINONA/common_bioinformatics/database/hgtdb_20230102/taxdump

In [ ]:
/data3/WINONA3/202208_exxon_sepsamples/10_trimming/binsanity_sample06.txt.002_contigs

grf-main \
    -i 10_trimming/binsanity_sample06.txt.002_contigs/Utg323392.fa \
    -o 09_MAGcompare/binsanity_06_002_grf \
    -c 0 --min_tr 10 --max_space 15000 -t 32
